In [151]:
import pandas as pd
import json

In [152]:
admins = json.load(open('spb_admin_areas.geojson', 'r'))

In [157]:
df_adm = pd.DataFrame([f['properties'] for f in admins['features'] if f['properties']['ADMIN_LVL'] == '8'])

In [159]:
df_adm.set_index('NAME')

,OSM_ID,ADMIN_LVL
NAME,,
Смолячково,-1120127.0,8
Молодёжное,-1120126.0,8
Серово,-1120125.0,8
Кронштадт,-1185387.0,8
Комарово,-1117563.0,8
...,...,...
Колпино,-337423.0,8
Металлострой,-337850.0,8
Усть-Ижора,-337851.0,8


In [21]:
df = pd.read_csv('20201020-locations.csv', usecols=['name', 'count'])

In [22]:
df

,name,count
0,КАД,653
1,Московское шоссе,494
2,Ленинский,422
3,Выборгское шоссе,413
4,Софийская,407
...,...,...
10468,Софийское шоссе,1
10469,Толмачово-Волосово,1
10470,Цивик Объезжайте,1
10471,Мечникова,1


In [25]:
df[df['count'] > 1]

,name,count
0,КАД,653
1,Московское шоссе,494
2,Ленинский,422
3,Выборгское шоссе,413
4,Софийская,407
...,...,...
3761,Бокситогорская улицы,2
3762,Огородного пер,2
3763,Копорское шоссе,2
3764,Семерка,2


In [101]:
def get_geolocation_data(data):
    r = data.get('raw', {})
    return {
        'address': loc.get('address'),
        'city': loc.get('city'),
        'quality': loc.get('quality'),
        'lat': loc.get('lat'),
        'lng': loc.get('lng'),
        'relevance': r.get('relevance'),
        'locality': r.get('locality'),
        'accuracy': r.get('accuracy'),
        'category': r.get('category'),
    }

In [105]:
import geocoder

# g = geocoder.yandex('Россия Санкт-Петербург Коломенская ул. 9')
# g = geocoder.yandex('Россия, Санкт-Петербург, Комсомола ул. 12к2')
# g = geocoder.google('Mountain View, CA')
# >>> g.latlng
mapbox_key = 'pk.eyJ1IjoidG1zaHYiLCJhIjoiM3BMLVc2MCJ9.PM9ukwAm-YUGlrBqt4V6vw'
def geocode(query):
    try:
        g = geocoder.mapbox(f'Санкт-Петербург, {query}', key=mapbox_key)
    #     g = geocoder.yandex(query)
    #     g = geocoder.google(query)
    #                         , method='reverse')
    #     g_loc = [g.json['lat'], g.json['lng']]
    #     print(g.json)
    #     return g.json
        return {
            **get_geolocation_data(g.json),
            'query': query,
        }
    except:
        return {
            'query': query,
        }
#     return g.latlng
#     return g.json['address']

# df_geocode = df.apply(geocode, axis=1)

In [44]:
loc = geocode('Московское шоссе')

In [35]:
loc = geocode('Выборгское шоссе')

In [56]:
loc = geocode('Большая Московская')

In [66]:
loc = geocode('Московская вставшие')

In [88]:
loc = geocode('Каршеринг')

In [106]:
locs = []
for x in df['name']:
    loc = geocode(x)
    locs.append(loc)

Status code Unknown from https://api.mapbox.com/geocoding/v5/mapbox.places/Санкт-Петербург, Репищевой.json: ERROR - HTTPSConnectionPool(host='api.mapbox.com', port=443): Read timed out.
Status code Unknown from https://api.mapbox.com/geocoding/v5/mapbox.places/Санкт-Петербург, Волхонском шоссе.json: ERROR - HTTPSConnectionPool(host='api.mapbox.com', port=443): Read timed out.
Status code 404 from https://api.mapbox.com/geocoding/v5/mapbox.places/Санкт-Петербург, Бухарестская/Белы Кун.json: ERROR - 404 Client Error: Not Found for url: https://api.mapbox.com/geocoding/v5/mapbox.places/%D0%A1%D0%B0%D0%BD%D0%BA%D1%82-%D0%9F%D0%B5%D1%82%D0%B5%D1%80%D0%B1%D1%83%D1%80%D0%B3,%20%D0%91%D1%83%D1%85%D0%B0%D1%80%D0%B5%D1%81%D1%82%D1%81%D0%BA%D0%B0%D1%8F/%D0%91%D0%B5%D0%BB%D1%8B%20%D0%9A%D1%83%D0%BD.json?access_token=pk.eyJ1IjoidG1zaHYiLCJhIjoiM3BMLVc2MCJ9.PM9ukwAm-YUGlrBqt4V6vw
Status code 404 from https://api.mapbox.com/geocoding/v5/mapbox.places/Санкт-Петербург, Дыбенко/Большевики.json: ERROR - 

In [102]:
geocode('Каршеринг')

{'address': 'Санкт-Петербу́рг, Saint Petersburg, Russia',
 'city': None,
 'quality': 0.574074,
 'lat': 59.95,
 'lng': 30.31667,
 'relevance': 0.574074,
 'locality': None,
 'accuracy': None,
 'category': None,
 'query': 'Каршеринг'}

In [108]:
df_loc = pd.DataFrame(locs)

In [109]:
df_loc.to_csv('20201020-geocoded.csv')

In [172]:
mask = df_loc['locality'].str.contains('Сенной', na=False)
df_loc.loc[mask, 'locality'] = 'Сенной округ'

In [180]:
mask = df_loc['locality'].str.contains('Адмиралтейский', na=False)
df_loc.loc[mask, 'locality'] = 'Адмиралтейский округ'

In [139]:
mask = (df_loc['relevance'] >= 0.7) & df_loc['locality'].notnull()
# has_coord = df_loc['lat'].notnull() & df_loc['lng'].notnull()
df_loc[mask]['locality'].value_counts().
# .set_index('locality')
# ['locality'].value_counts().rename_axis('locality').reset_index(name='count')

округ Сенной округ            374
округ Смольнинское            214
округ Кронверкское            204
округ Звездное                181
округ Адмиралтейский округ    162
                             ... 
Ушково                          3
Комарово                        2
Солнечное                       2
Смолячково                      1
Молодежное                      1
Name: locality, Length: 111, dtype: int64

In [211]:
mask = (df_loc['relevance'] >= 0.7) & df_loc['locality'].notnull()
has_coord = df_loc['lat'].notnull() & df_loc['lng'].notnull()
df_loc[mask & has_coord]
# .set_index('locality')
# ['locality'].value_counts().rename_axis('locality').reset_index(name='count')

,address,city,quality,lat,lng,relevance,locality,accuracy,category,query
0,"Санкт-Петербу́рг, Saint Petersburg, Russia",NaN,0.574074,59.95,30.31667,1.000000,округ Екатерингофский,NaN,NaN,КАД
1,"Санкт-Петербу́рг, Saint Petersburg, Russia",NaN,0.574074,59.95,30.31667,1.000000,округ Звездное,NaN,NaN,Московское шоссе
2,"Санкт-Петербу́рг, Saint Petersburg, Russia",NaN,0.574074,59.95,30.31667,1.000000,округ Измайловское,NaN,NaN,Ленинский
3,"Санкт-Петербу́рг, Saint Petersburg, Russia",NaN,0.574074,59.95,30.31667,1.000000,округ Шувалово-Озерки,NaN,NaN,Выборгское шоссе
4,"Санкт-Петербу́рг, Saint Petersburg, Russia",NaN,0.574074,59.95,30.31667,1.000000,75-й муниципальный округ Санкт-Петербурга,NaN,NaN,Софийская
...,...,...,...,...,...,...,...,...,...,...
217,"Санкт-Петербу́рг, Saint Petersburg, Russia",NaN,0.574074,59.95,30.31667,1.000000,округ Черная речка,NaN,NaN,Приморский проспект
218,"Санкт-Петербу́рг, Saint Petersburg, Russia",NaN,0.574074,59.95,30.31667,1.000000,округ Шувалово-Озерки,NaN,NaN,Выборгское
221,"Санкт-Петербу́рг, Saint Petersburg, Russia",NaN,0.574074,59.95,30.31667,0.703704,округ Литейный округ,NaN,NaN,Перекресток пр
222,"Санкт-Петербу́рг, Saint Petersburg, Russia",NaN,0.574074,59.95,30.31667,1.000000,65-й муниципальный округ Санкт-Петербурга,NaN,NaN,Савушкин


In [202]:
def calc_adm_coords(df):
    adm_coords = {}
    for _, row in df.iterrows():
        key = row['locality']
        if not key in adm_coords or not adm_coords[key]:
            adm_coords[key] = (row['lng'], row['lat'])
    return adm_coords

In [208]:
mask = (df_loc['relevance'] >=0.7) & df_loc['lat'].notnull() & df_loc['lng'].notnull() & df_loc['locality'].notnull()
adm_coords = calc_adm_coords(df_loc[mask])

mask = (df_loc['relevance'] >=0.7) & df_loc['locality'].notnull()
df_result = df_loc[mask]['locality'].value_counts().rename_axis('locality').reset_index(name='count')

df_result['lng'] = df_result.apply(lambda x: adm_coords.get(x['locality'], (None, None))[0], axis=1)
df_result['lat'] = df_result.apply(lambda x: adm_coords.get(x['locality'], (None, None))[1], axis=1)

df_result = df_result[df_result['lat'].notnull() & df_result['lng'].notnull()]
df_result

,locality,count,lng,lat
0,Сенной округ,374,30.31667,59.95
1,округ Смольнинское,214,30.31667,59.95
2,округ Кронверкское,204,30.31667,59.95
3,округ Звездное,181,30.31667,59.95
4,Адмиралтейский округ,162,30.31667,59.95
...,...,...,...,...
91,округ Сосновая поляна,17,30.31667,59.95
92,Репино,16,30.31667,59.95
93,округ Балканский,15,30.31667,59.95
94,округ Купчино,15,30.31667,59.95


In [220]:
x = df_result.head(30).to_markdown()
print(x)

|    | locality                                  |   count |     lng |   lat |
|---:|:------------------------------------------|--------:|--------:|------:|
|  0 | Сенной округ                              |     374 | 30.3167 | 59.95 |
|  1 | округ Смольнинское                        |     214 | 30.3167 | 59.95 |
|  2 | округ Кронверкское                        |     204 | 30.3167 | 59.95 |
|  3 | округ Звездное                            |     181 | 30.3167 | 59.95 |
|  4 | Адмиралтейский округ                      |     162 | 30.3167 | 59.95 |
|  5 | округ Ржевка                              |     158 | 30.3167 | 59.95 |
|  6 | Пушкин                                    |     152 | 30.3167 | 59.95 |
|  7 | Колпино                                   |     141 | 30.3167 | 59.95 |
|  8 | округ Московская застава                  |     135 | 30.3167 | 59.95 |
|  9 | округ Чкаловское                          |     132 | 30.3167 | 59.95 |
| 10 | округ Литейный округ                      |  

In [209]:
df_result.to_csv('20201020-vk-spb_today-adm_stat.csv')

In [197]:
df_result.apply(lambda x: adm_coords[x['locality']][0], axis=1)

KeyError: 'округ Большая Охта'

In [179]:
df_adm[df_adm['NAME'].str.contains('Адмиралтейский')]

,OSM_ID,NAME,ADMIN_LVL
56,-1123300.0,Адмиралтейский округ,8


In [183]:
df_result['osm_id'] = df_adm.set_index('NAME')['OSM_ID']
# = df_result.set_index('locality')['count']
df_result[df_result['osm_id'].isnull()]

,count,osm_id
locality,,
округ Звездное,181,NaN
округ Литейный округ,127,NaN
округ Дворцовый округ,123,NaN
округ Финляндский округ,117,NaN
65-й муниципальный округ Санкт-Петербурга,112,NaN
75-й муниципальный округ Санкт-Петербурга,106,NaN
округ Введенский,92,NaN
округ Владимирский округ,81,NaN
округ Екатерингофский,76,NaN


In [149]:
mask = (df_loc['locality'] == 'округ Сенной округ') & (df_loc['relevance'] == 1)
df_loc[mask]

,address,city,quality,lat,lng,relevance,locality,accuracy,category,query
66,"Санкт-Петербу́рг, Saint Petersburg, Russia",NaN,0.574074,59.95,30.31667,1.0,округ Сенной округ,NaN,NaN,Фонтанка
131,"Санкт-Петербу́рг, Saint Petersburg, Russia",NaN,0.574074,59.95,30.31667,1.0,округ Сенной округ,NaN,NaN,Перекресток
700,None,NaN,NaN,NaN,NaN,1.0,округ Сенной округ,NaN,NaN,Исаакиевская площадь
752,None,NaN,NaN,NaN,NaN,1.0,округ Сенной округ,NaN,NaN,Сенная площадь
860,None,NaN,NaN,NaN,NaN,1.0,округ Сенной округ,NaN,NaN,Казанская
916,None,NaN,NaN,NaN,NaN,1.0,округ Сенной округ,NaN,NaN,Вознесенский проспект
970,None,NaN,NaN,NaN,NaN,1.0,округ Сенной округ,NaN,NaN,Сенная
1071,None,NaN,NaN,NaN,NaN,1.0,округ Сенной округ,NaN,NaN,Сенной
1320,None,NaN,NaN,NaN,NaN,1.0,округ Сенной округ,NaN,NaN,Обуховская
1370,None,NaN,NaN,NaN,NaN,1.0,округ Сенной округ,NaN,NaN,Вознесенский пр


In [144]:
# df_coords = df_loc[['locality', 'lat', 'lng']]
# df_result['lat'] = df_coords
# df_coords[df_result['locality'] == df_loc['locality']]
df_result['locality'].apply(lambda x: df_loc['locality'])

ValueError: Can only compare identically-labeled Series objects

In [143]:
# left = df_loc[['locality', 'lat', 'lng']]
# right = df_result
# left.merge(right, left_on='locality', right_on='locality', how='inner')

,locality,lat,lng,count
0,округ Екатерингофский,59.95,30.31667,76
1,округ Екатерингофский,59.95,30.31667,76
2,округ Екатерингофский,59.95,30.31667,76
3,округ Екатерингофский,59.95,30.31667,76
4,округ Екатерингофский,NaN,NaN,76
...,...,...,...,...
7632,Песочный,NaN,NaN,5
7633,Песочный,NaN,NaN,5
7634,Песочный,NaN,NaN,5
7635,Смолячково,NaN,NaN,1
